In [62]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [63]:
df = pd.read_csv('drinking_water_potability.csv')
df

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890456,20791.31898,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.05786,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.54173,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.41744,8.059332,356.886136,363.266516,18.436525,100.341674,4.628771,0
4,9.092223,181.101509,17978.98634,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
...,...,...,...,...,...,...,...,...,...,...
3271,4.668102,193.681736,47580.99160,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821,1
3272,7.808856,193.553212,17329.80216,8.061362,NaN,392.449580,19.903225,NaN,2.798243,1
3273,9.419510,175.762646,33155.57822,7.350233,NaN,432.044783,11.039070,69.845400,3.298875,1
3274,5.126763,230.603758,11983.86938,6.303357,NaN,402.883113,11.168946,77.488213,4.708658,1


In [64]:
df["Missing"] = df.isna().any(axis=1)
dimensions = ["ph",
              "Hardness",
              "Solids",
              "Chloramines",
              "Sulfate",
              "Conductivity",
              "Organic_carbon",
              "Trihalomethanes",
              "Turbidity"]

# Gestion des valeurs manquantes : Simple Imputer


In [65]:
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer

In [66]:
# 1 : Suppression de toutes les lignes avec 2 ou 3 valeurs manquantes et imputation pour les lignes avec 1 valeur manquante
df_missing_1 = df.dropna(thresh=10)

In [67]:
# 1.1 : Mean-imputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(df_missing_1)
df_missing_11 = pd.DataFrame(imp_mean.transform(df_missing_1))

df_missing_11.head()

,0,1,2,3,4,5,6,7,8,9,10
0,7.078708,204.890456,20791.31898,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0.0,1.0
1,3.716080,129.422921,18630.05786,6.635246,333.699670,592.885359,15.180013,56.329076,4.500656,0.0,1.0
2,8.099124,224.236259,19909.54173,9.275884,333.699670,418.606213,16.868637,66.420093,3.055934,0.0,1.0
3,8.316766,214.373394,22018.41744,8.059332,356.886136,363.266516,18.436525,100.341674,4.628771,0.0,0.0
4,9.092223,181.101509,17978.98634,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0.0,0.0


In [68]:
# 1.2 : Median-imputer
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
imp_median.fit(df_missing_1)
df_missing_12 = pd.DataFrame(imp_median.transform(df_missing_1))


In [69]:
# 2 : Suppression des lignes avec 3 valeurs manquantes et imputation pour les autres
df_missing_2 = df.dropna(thresh=9)


In [70]:
# 2.1 : Mean
imp_mean.fit(df_missing_2)
df_missing_21 = pd.DataFrame(imp_mean.transform(df_missing_2))


In [71]:
# 2.2 : Median
imp_median.fit(df_missing_2)
df_missing_22 = pd.DataFrame(imp_median.transform(df_missing_2))

In [72]:
#Dictionnaire qui contient les dataframes d'intérêt
all_df_missing = {"df_missing_11": df_missing_11, "df_missing_12": df_missing_12,
                  "df_missing_21": df_missing_21, "df_missing_22": df_missing_22}

for dataframe in all_df_missing.values():
    #on redonne d'abord les noms des colonnes qui ont disparu
    dataframe.columns = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
                         'Organic_carbon', 'Trihalomethanes', 'Turbidity', 'Potability',
                         'Missing']
    dataframe.drop('Missing', axis=1, inplace=True)
    #fig = px.scatter(y=dataframe.loc[:,"ph"])
    dataframe.columns
    #fig.show()

On se rend compte avec ces imputations, du fait que la méthode "most frequent value" n'est pas adaptée à notre dataset. en effet, la valeur la plus fréquente est un outlier (0 pour le ph par exemple). 
On se concentre donc sur la valeur moyenne et la médiane, et on abandonne la "most frequent value"

In [73]:
df_test = all_df_missing["df_missing_22"]
df_test.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,7.036752,204.890456,20791.31898,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0.0
1,3.716080,129.422921,18630.05786,6.635246,333.073546,592.885359,15.180013,56.329076,4.500656,0.0
2,8.099124,224.236259,19909.54173,9.275884,333.073546,418.606213,16.868637,66.420093,3.055934,0.0
3,8.316766,214.373394,22018.41744,8.059332,356.886136,363.266516,18.436525,100.341674,4.628771,0.0
4,9.092223,181.101509,17978.98634,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0.0


# Iterative Imputer

### Nous allons ici utiliser une imputation multivariable sur les données ayant max 2 valeurs manquantes.

In [74]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

df_missing_3 = df.dropna(thresh=10)
imp_iter = IterativeImputer(random_state=0)
df_missing_3 = pd.DataFrame(imp_iter.fit_transform(df_missing_3))
df_missing_3.columns = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
                        'Organic_carbon', 'Trihalomethanes', 'Turbidity', 'Potability',
                        'Missing']
df_missing_3.drop('Missing', axis=1, inplace=True)
all_df_missing["df_missing_3"] = df_missing_3


## A ce stade, on a donc 7 datasets dont les valeurs manquantes ont été imputées selon différentes méthodes. Nous allons évaluer nos modèles sur chacun de ces datasets et garder celui pour lesquels les résultats sont les plus probants.

# Best Subset Selection


In [75]:
# Subset Selection
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.metrics import mean_squared_error
import itertools
import statsmodels.api as sm

# Some libraries for PCA visualization
import seaborn as sns
#Make Plotly figure
import plotly.graph_objs as go

y = df_missing_11['Potability']  #mettre le bon df
X = df_missing_11.drop('Potability', axis=1)


def processSubset(feature_set):
    # Fit OLS (Ordinary Least Squares) model on feature_set and calculate RSS
    model = sm.OLS(y, X[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
    return {"model": regr, "RSS": RSS}


def getBest(k):
    results = []

    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]

    print("Processed", models.shape[0], "models on", k, "predictors ")

    # Return the best model, along with some other useful information about the model
    return best_model


# It might take a while... Please be patient. 
# As long as the star to the left of the cell is displayed do not run again the cell.

models_best = pd.DataFrame(columns=["RSS", "model"])

for i in range(1, 10):
    models_best.loc[i] = getBest(i)



Processed 9 models on 1 predictors 
Processed 36 models on 2 predictors 
Processed 84 models on 3 predictors 
Processed 126 models on 4 predictors 
Processed 126 models on 5 predictors 
Processed 84 models on 6 predictors 
Processed 36 models on 7 predictors 
Processed 9 models on 8 predictors 
Processed 1 models on 9 predictors 


In [76]:
print(models_best.loc[5, "model"].summary())

                                 OLS Regression Results                                
Dep. Variable:             Potability   R-squared (uncentered):                   0.392
Model:                            OLS   Adj. R-squared (uncentered):              0.391
Method:                 Least Squares   F-statistic:                              400.8
Date:                Wed, 03 Nov 2021   Prob (F-statistic):                        0.00
Time:                        23:56:03   Log-Likelihood:                         -2187.0
No. Observations:                3116   AIC:                                      4384.
Df Residuals:                    3111   BIC:                                      4414.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

# Définition de la pipeline

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

# Each object of the pipeline is identified by a key ('std_scaler', 'linreg').
# You can use any value as the key of an object in the pipeline
pipeline = Pipeline([
    ('iterative_imputer', IterativeImputer(sample_posterior=True)),
    ('std_scaler', StandardScaler()),
    ('linreg', SGDRegressor())
])

# Train_test_split

In [78]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

test_size = 0.2
y = df['Potability']  #mettre le bon df
X = df.drop('Potability', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
# The function fit is used to train the linear regressor on the training data.
# The result is a line with a given intercept and slope.
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("The prediction error (RMSE) is {}".format(np.sqrt(mse)))

The prediction error (RMSE) is 0.4841371333440589


# Model Testing


In [79]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
import plotly.express as px
import pandas as pd
import sklearn.tree as tree

stats = pd.DataFrame()


def computeModelStats(model, stats, dataset, hyperParams, randomizedSearch=False):
    """
    :param model: model that will be assessed
    :type model: {tuple(predictor,name:str)}
    :param stats: Dataframe in which results will be sent
    :type stats: {Dataframe}
    :param dataset: Dataset that will be used
    :type dataset: {tuple(Dict(X:DataFrame,y:array), name)}
    :param hyperParams: HyperParams grid that will be assessed
    :type hyperParams: {Dict}
    :return: void
    :rtype: void
    """
    X = dataset[0]["X"]
    y = dataset[0]["y"]

    if not randomizedSearch:
        grid_search = GridSearchCV(model[0], cv=StratifiedKFold(n_splits=5), refit='f1',
                                   scoring=['f1', 'recall', 'precision'], verbose=3, n_jobs=-1, param_grid=hyperParams,
                                   return_train_score=True)
    else:
        grid_search = RandomizedSearchCV(model[0], cv=StratifiedKFold(n_splits=5), refit='f1',
                                         scoring=['f1', 'recall', 'precision'], verbose=3, n_jobs=-1,
                                         param_distributions=hyperParams, return_train_score=True,
                                         n_iter=randomizedSearch)

    grid_search.fit(X, y)

    temp_test = pd.DataFrame({key.replace("_test", ""): grid_search.cv_results_[key] for key in
                              ["mean_test_f1", "std_test_f1", "mean_test_recall", "mean_test_precision", "params"]})
    temp_train = pd.DataFrame({key.replace("_train", ""): grid_search.cv_results_[key] for key in
                               ["mean_train_f1", "std_train_f1", "mean_train_recall", "mean_train_precision",
                                "std_test_f1", "params"]})
    temp_train["type"] = "train"
    temp_test["type"] = "test"
    temp = temp_test.append(temp_train)
    temp["dataset"] = dataset[1]
    temp["model"] = model[1]
    temp["params"] = temp["params"].astype(str)
    stats = stats.append(temp)
    fig = px.bar(temp, y="mean_f1", hover_data=["params"], barmode="group", hover_name="params", color="type",
                  title=f"Mean f1 score on testing and training set for each parameter combination <br> (Model: {model[1]}, Dataset: {dataset[1]})",
                  labels={"mean_f1": f"Average f1", "index": "Params"}, error_y="std_f1")

    fig.show()
    return stats


def extract_features_and_labels(df):
    print(df.columns)
    y = df.Potability
    X = df.drop("Potability", axis=1)
    return {"X": X, "y": y}


In [80]:
mean_accurracy_3_missing_values = (
    extract_features_and_labels(df_missing_11), "Mean imputation for individuals with 3 MV")

mean_accurracy_2_3_missing_values = (
    extract_features_and_labels(df_missing_21), "Mean imputation for individuals with 2 or 3 MV")

multivariate_feature_imputation = (extract_features_and_labels(df_missing_3), "Multivariate feature imputation")

multivariate_feature_imputation_subset_selection = (
    extract_features_and_labels(df_missing_3[["ph", "Hardness", "Chloramines", "Sulfate", "Potability"]]),
    "Multivariate feature imputation with subset selection")

mean_accurracy_2_3_missing_values_subset_selection = (
    extract_features_and_labels(df_missing_21[["ph", "Hardness", "Chloramines", "Sulfate", "Potability"]]),
    "Mean imputation subset selection for individuals with 2 or 3 MV")

Index(['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity', 'Potability'],
      dtype='object')
Index(['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity', 'Potability'],
      dtype='object')
Index(['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity', 'Potability'],
      dtype='object')
Index(['ph', 'Hardness', 'Chloramines', 'Sulfate', 'Potability'], dtype='object')
Index(['ph', 'Hardness', 'Chloramines', 'Sulfate', 'Potability'], dtype='object')


# Random Forest
We know usually we tend to end with random forest. However we wanted to test several datasets, given the ability of random forests to ability to match unscaled data it will help us to prune some dataset for the next models.

In [81]:
from sklearn.ensemble import RandomForestClassifier
model = (RandomForestClassifier(), "RandomForest Classifier")

pgrid = {"max_depth": [3,6,9],
         "n_estimators": [50, 80, 110, 130],
         "min_samples_split": [3, 6, 9, 12],
         "max_features": ["sqrt", "log2"]}


stats = computeModelStats(model, stats, mean_accurracy_3_missing_values, pgrid)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


In [82]:
stats = computeModelStats(model, stats, mean_accurracy_2_3_missing_values, pgrid)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


In [83]:
stats = computeModelStats(model, stats, mean_accurracy_2_3_missing_values_subset_selection, pgrid)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


In [84]:
stats = computeModelStats(model, stats, multivariate_feature_imputation, pgrid)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


In [85]:
stats = computeModelStats(model, stats, multivariate_feature_imputation_subset_selection, pgrid)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


In [89]:
def plot_params_incidence(model=None,dataset=None,params=None,barchart=False):
    mask1 = stats["dataset"] == dataset[1] if dataset != None else True
    mask2 = stats["model"] == model[1] if model != None else True
    mask3 = stats["type"] == "test"

    df_stats_temp = stats[mask1&mask2&mask3]
    df_stats_temp = df_stats_temp.join(pd.json_normalize(df_stats_temp["params"].apply(eval))).groupby([params]).agg({"mean_f1":"mean","std_f1":"mean"})

    if not barchart:
        return px.line(df_stats_temp, labels="params",y="mean_f1",title=f"Mean f1 score on testing depending on hyper parameter values <br>(Model: {model[1] if model else 'All'}, Dataset: {dataset[1]if dataset else 'All'}").show()
    return px.bar(df_stats_temp, labels="params",y="mean_f1",title=f"Mean f1 score on testing depending on hyper parameter values <br>(Model: {model[1] if model else 'All'}, Dataset: {dataset[1]if dataset else 'All'})",error_y="std_f1").show()

In [91]:
plot_params_incidence(model=model,params = "max_depth", barchart=True)

In [94]:
plot_params_incidence(model=model,params = "n_estimators", barchart=True)

In [96]:
plot_params_incidence(model=model,params = "min_samples_split", barchart=True)

In [97]:
plot_params_incidence(model=model,params = "max_features", barchart=True)